In [0]:
!git clone https://github.com/pjreddie/darknet

In [0]:
cd darknet

In [0]:

!sed -i 's/GPU=0/GPU=1/g' Makefile
!make
!wget https://pjreddie.com/media/files/yolov3.weights
!pip install opencv-python  

In [0]:
cd '/content'

In [0]:


#-------------------------------------------------------------------------------------------------------------------------------
from IPython.display import Image
#-------------------------------------------------------------------------------------------------------------------------------
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import IPython
from google.colab.patches import cv2_imshow
#-------------------------------------------------------------------------------------------------------------------------------
# load the COCO class labels our YOLO model was trained on
LABELS = open("/content/darknet/data/coco.names").read().strip().split("\n")

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet("/content/darknet/cfg/yolov3.cfg", "/content/darknet/yolov3.weights")

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [0]:
import matplotlib.pyplot as plt

from skimage import data
from skimage.color import rgb2hsv
from skimage import io


In [0]:
def detect_teams(img):
  pixels = np.float32(img.reshape(-1, 3))
  n_colors = 2
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
  flags = cv2.KMEANS_RANDOM_CENTERS
  _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
  _, counts = np.unique(labels, return_counts=True)
   
  return palette[np.argmax(counts)]

In [0]:

filename = "1.jpg"
img = io.imread(filename)[:, :, :-1]

imageSource = '/content/'+filename

img = cv2.imread(imageSource)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

# grab the image dimensions and convert it to a blob
(H, W) = img.shape[:2]
# construct a blob from the input image and then perform a forward
# pass of the YOLO object detector, giving us our bounding boxes and
# associated probabilities
blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
swapRB=True, crop=False)
net.setInput(blob)
layerOutputs = net.forward(ln)

# initialize our lists of detected bounding boxes, confidences, and
# class IDs, respectively
boxes = []
confidences = []
classIDs = []
centers = []
detected= []


# loop over each of the layer outputs
for output in layerOutputs:

  # loop over each of the detections
  for detection in output:

      # extract the class ID and confidence (i.e., probability) of
      # the current object detection
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]

      # filter out weak predictions by ensuring the detected
      # probability is greater than the minimum probability
      #and filter out classes we haven't measure yet    
      if confidence > 0.75:  #and classID in ourClassesIDs:

          # scale the bounding box coordinates back relative to the
          # size of the image, keeping in mind that YOLO actually
          # returns the center (x, y)-coordinates of the bounding
          # box followed by the boxes' width and height
          box = detection[0:4] * np.array([W, H, W, H])
          (centerX, centerY, width, height) = box.astype("int")

          # use the center (x, y)-coordinates to derive the top and
          # and left corner of the bounding box
          x = int(centerX - (width / 2))
          y = int(centerY - (height / 2))

          # update our list of bounding box coordinates, confidences,
          # and class IDs
          boxes.append([x, y, int(width), int(height)])
          confidences.append(float(confidence))
          classIDs.append(classID)
          centers.append((centerX, centerY))
#--------------------------------------------------------------------------------------------------          

# apply non-maxima suppression to suppress weak, overlapping
# bounding boxes
idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

players_colors = []
players_imgs = []
# ensure at least one detection exists
if len(idxs) > 0:

  # loop over the indexes we are keeping
  for i in idxs.flatten():
    # extract the bounding box coordinates
    (x, y) = (boxes[i][0], boxes[i][1])
    (w, h) = (boxes[i][2], boxes[i][3])

    if LABELS[classIDs[i]] == "person": #or LABELS[classIDs[i]] == 'sports ball' :
      # draw a bounding box rectangle and label on the frame


      color = [int(c) for c in COLORS[classIDs[i]]]
      cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
      player_img = img[y:y+h,x:x+w]

      player_color = detect_teams(player_img)

      players_imgs.append(player_img)

      players_colors.append(player_color)
      

        
      


cv2_imshow(img)

for i in range(len(players_imgs)):

  #print(color_difference(players_colors[6], players_colors[i]))
  print(players_colors[i])
  cv2_imshow(players_imgs[i])
  


In [0]:
  cv2.imwrite("p0.jpg", players_imgs[0])

In [0]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
image = cv2.imread( "/content/p0.jpg")
#image = players_imgs[0]
count = 0
success = True
idx = 0
hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
#green range
lower_green = np.array([40,40, 40])
upper_green = np.array([70, 255, 255])
#Define a mask ranging from lower to uppper
mask = cv2.inRange(hsv, lower_green, upper_green)
#Do masking
res = cv2.bitwise_and(image, image, mask=mask)
#convert to hsv to gray
res_bgr = cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
res_gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)

cv2.imwrite("mask.jpg", res_bgr)


from PIL import Image


mask_image = Image.open("mask.jpg")
mask_pixels = mask_image.load()

org_image = Image.open("p0.jpg")
org_pixels = org_image.load()

w, h = org_image.size


for i in range(w):
  for j in range(h):
    if mask_pixels[i,j] != (0, 0, 0):
      org_pixels[i, j] = (0, 0, 0)
      

org_image.save("mp0.jpg")      
#cv2_imshow("mp0.jpg")

In [0]:
f = "/content/mp0.jpg"

cv2_imshow(cv2.imread(f))

In [0]:
print(detect_teams(cv2.imread(f)))

In [0]:
from __future__ import print_function
import binascii
import struct
from PIL import Image
import numpy as np
import scipy
import scipy.misc
import scipy.cluster

NUM_CLUSTERS = 5

print('reading image')
im = Image.open('mp0.jpg')
#im = im.resize((150, 150))      # optional, to reduce time
ar = np.asarray(im)
shape = ar.shape
ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)

print('finding clusters')
codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
print('cluster centres:\n', codes)

vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences

index_max = scipy.argmax(counts)                    # find most frequent
peak = codes[index_max]
colour = binascii.hexlify(bytearray(int(c) for c in peak)).decode('ascii')
print('most frequent is %s (#%s)' % (peak, colour))

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io



c = players_colors[3]
print(c[0])
palette = np.array([[c[0],   c[1],   c[2]], # index 0: red
                    [  0, 255,   0], # index 1: green
                    ], dtype=np.uint8)

m, n = 1, 1
indices = np.random.randint(0, len(palette), size=(1, 1))
indices[0] = 0
io.imshow(palette[indices])

In [0]:
 import matplotlib.pyplot as plt
for p in players_colors:
  
  plt.imshow(p.reshape(3,1))
  plt.show()
  
  

In [0]:
 
image = cv2.imread( "/content/3.jpg")
count = 0
success = True
idx = 0

In [0]:
#Read the video frame by frame

#converting into hsv image
hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
#green range
lower_green = np.array([40,40, 40])
upper_green = np.array([70, 255, 255])
#blue range
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])
#Red range
lower_red = np.array([0,31,255])
upper_red = np.array([176,255,255])
#white range
lower_white = np.array([0,0,0])
upper_white = np.array([0,0,255])

In [0]:
#Define a mask ranging from lower to uppper
 mask = cv2.inRange(hsv, lower_red, upper_red)
 #Do masking
 res = cv2.bitwise_and(image, image, mask=mask)
 #convert to hsv to gray
 res_bgr = cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
 res_gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)

In [0]:
cv2_imshow(res_bgr)

In [0]:
print(dominant)

In [0]:
!sudo pip install colormath

In [0]:
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

c1 = players_colors[0]




color1_rgb = sRGBColor(c1[0], c1[1], c1[2]);
color1_lab = convert_color(color1_rgb, LabColor);

for c in players_colors:

  color2_rgb = sRGBColor(c[0], c[1], c[2]);
  color2_lab = convert_color(color2_rgb, LabColor);


  delta_e = delta_e_cie2000(color1_lab, color2_lab);
  print ("The difference between the 2 color = ", delta_e)
  print("--------------------------------------------------------------------")










In [0]:
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000




def color_difference(c1, c2):
  
  color1_rgb = sRGBColor(c1[0], c1[1], c1[2]);
  color1_lab = convert_color(color1_rgb, LabColor);

  color2_rgb = sRGBColor(c2[0], c2[1], c2[2]);
  color2_lab = convert_color(color2_rgb, LabColor);


  delta_e = delta_e_cie2000(color1_lab, color2_lab);

  return delta_e
  


    







